In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-y498m7e1
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-y498m7e1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=77bb56f6ee83ecbe873973f28e9f67575c2250d68f6373c85e80d26887b0948a
  Stored in directory: /tmp/pip-ephem-wheel-cache-2z52oap8/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:

%%cu
#include < string>
#include < fstream>
#include < cstdio>
#include < iostream>

///////////////////////////////////////////////////////////////////////////////
// FINISHED KERNELS (you dont have to change anything)
///////////////////////////////////////////////////////////////////////////////

// safe division
#define SDIV(x,y)(((x)+(y)-1)/(y))

template <
    typename index_t,
    typename value_t>
void load_binary(
    const value_t * data,
    const index_t length,
    std::string filename) {

    std::ifstream ifile(filename.c_str(), std::ios::binary);

    if(!ifile.good()) {
        throw std::runtime_error{"can't open file " + filename};
    }

    ifile.read((char*) data, sizeof(value_t)*length);
    ifile.close();
}

template <
    class index_t,
    class value_t>
__global__
void compute_mean_kernel(
    const value_t * data,
    value_t * mean,
    const index_t num_entries,
    const index_t num_features)
{
    const auto thid = blockDim.x*blockIdx.x + threadIdx.x;

    if (thid < num_features) {
        value_t accum = 0;

        for (index_t entry = 0; entry < num_entries; entry++)
            accum += data[entry*num_features+thid];

        mean[thid] = accum/num_entries;
    }
}

template <
    class index_t,
    class value_t>
__global__
void correction_kernel(
    value_t * data,
    const value_t * mean,
    const index_t num_entries,
    const index_t num_features)
{
    const auto thid = blockDim.x*blockIdx.x + threadIdx.x;

    if (thid < num_features) {
        const value_t value = mean[thid];

        for (index_t entry = 0; entry < num_entries; entry++)
            data[entry*num_features+thid] -= value;
    }
}

///////////////////////////////////////////////////////////////////////////////
// STUDENTS PART (fill in the gaps)
///////////////////////////////////////////////////////////////////////////////

template <
    class index_t,
    class value_t,
    uint32_t chunk_size = 32>
__global__
void shared_covariance_kernel(
    const value_t * data,
    value_t * cov,
    const index_t num_entries,
    const index_t num_features)
{
    // convenience variables
    const index_t base_x = blockIdx.x*chunk_size;
    const index_t base_y = blockIdx.y*chunk_size;

    const index_t thid_y = threadIdx.y;
    const index_t thid_x = threadIdx.x;

    const index_t x = base_x + thid_x;
    const index_t y = base_y + thid_y;

    // optional early exit: -500ms
    if (base_x > base_y) return;

    // allocate shared memory
    __shared__ value_t s_cache_x[chunk_size][chunk_size];
    __shared__ value_t s_cache_y[chunk_size][chunk_size];

    // compute the number of chunks to be computed
    const index_t num_chunks = SDIV(num_entries, chunk_size);

    // accumulated value of scalar product
    value_t accum = 0;

    // for each chunk
    for (index_t chunk = 0; chunk < num_chunks; chunk++) {
        // assign thread IDs to rows and columns
        const index_t row   = thid_y + chunk*chunk_size;
        const index_t col_x = thid_x + base_x;
        const index_t col_y = thid_x + base_y;

        // check if valid row or column indices
        const bool valid_row   = row   < num_entries;
        const bool valid_col_x = col_x < num_features;
        const bool valid_col_y = col_y < num_features;

        ///////////////////////////////////////////////////////////////////////
        // fill shared memory with tiles where thid_y enumerates
        // image identifiers (entries) and thid_x denotes feature
        // coordinates (pixels). s_cache_x corresponds to x and
        // s_cache_y to y where cov[x,y] is the pairwise covariance
        // FILL IN YOUR CODE HERE
        s_cache_x[thid_y][thid_x] = 0;
        s_cache_y[thid_y][thid_x] = 0;
       
        ///////////////////////////////////////////////////////////////////////

        // this is needed to ensure that all threads finished writing
        // to shared memory
        __syncthreads();

        // optional early exit: -100ms
        if (x <= y)
            // here we actually evaluate the scalar product
            for (index_t entry = 0; entry < chunk_size; entry++)
                accum += s_cache_y[entry][thid_y]*s_cache_x[entry][thid_x];

        // this is needed to ensure that shared memory can be over-
        // written again in the next iteration
        __syncthreads();
    }

    // since cov[x,y] = cov[y,x] we only compute one entry
    if (y < num_features && x <= y) {
        const value_t result = accum/num_entries;
        cov[y*num_features+x] = result;
        cov[x*num_features+y] = result;
    }
}

///////////////////////////////////////////////////////////////////////////////
// MAIN PROGRAM
///////////////////////////////////////////////////////////////////////////////

int main () {

    // choose GPU 0 (GTX 1080 (Pascal) 8GB RAM)
    // or GPU 1 (Titan X (Maxwell) 12GB RAM)
    cudaSetDevice(0);                                                     

    // 202599 grayscale images each of shape 55 x 45
    constexpr uint64_t num_images = 10000, num_rows = 55, num_cols = 45;
    constexpr uint64_t num_pixels = num_rows * num_cols;

    // pointer for data matrix and mean vector
    float * h_data = nullptr;
    float * h_cov = nullptr;
    cudaMallocHost(&h_data, sizeof(float)*num_images*num_pixels);         
    cudaMallocHost(&h_cov,  sizeof(float)*num_pixels*num_pixels);         

    // allocate storage on GPU
    float * d_data = nullptr;
    float * d_mean = nullptr;
    float * d_cov = nullptr;
    cudaMalloc(&d_data, sizeof(float)*num_images*num_pixels);            
    cudaMalloc(&d_mean, sizeof(float)*num_pixels);                       
    cudaMalloc(&d_cov,  sizeof(float)*num_pixels*num_pixels);           

    // load data matrix from disk
    std::string file_name = "/content/celebA_gray_lowres.10000_55_45_32.bin";
    load_binary(h_data, num_images*num_pixels, file_name);
  
    // copy data to device and reset d_mean
    cudaMemcpy(d_data, h_data, sizeof(float)*num_images*num_pixels,
               cudaMemcpyHostToDevice);                                  
    cudaMemset(d_mean, 0, sizeof(float)*num_pixels);                     

    // compute mean
     compute_mean_kernel<<< SDIV(num_pixels, 1024), 1024 >>>
                       (d_data, d_mean, num_images, num_pixels);                                      

    // correct mean
    correction_kernel<<< SDIV(num_pixels, 1024), 1024 >>>
                       (d_data, d_mean, num_images, num_pixels);       
 
    // compute covariance matrix
    const dim3 blocks(SDIV(num_pixels, 32), SDIV(num_pixels, 32));
    const dim3 threads(32, 32);
    shared_covariance_kernel<<< blocks, threads >>>
                       (d_data, d_cov, num_images, num_pixels);        
 
    // transfer covariance back to host
     cudaMemcpy(h_cov, d_cov, sizeof(float)*num_pixels*num_pixels,
               cudaMemcpyDeviceToHost);                                
     std::cout << '\n';

    // write mean image to disk
    float * h_cov_check = nullptr;
    cudaMallocHost(&h_cov_check, sizeof(float)*num_pixels*num_pixels);   
    load_binary(h_cov_check, num_pixels*num_pixels,
                         "/content/celebA_covariance.10000.bin");

    bool no_errors = true;
    for (uint64_t i = 0; i < num_pixels*num_pixels; i++) {
        if ((h_cov[i]-h_cov_check[i])*(h_cov[i]-h_cov_check[i]) > 1E-4) {
            std::cout << "ERROR: at position " << i << " of the COV matrix"
                      << std::endl;
            std::cout << "h: " << h_cov[i] << " check: " << h_cov_check[i]; 
            no_errors = false;
            break;
        }
    }

    // free allocated memory
    cudaFreeHost(h_data);                                                
    cudaFreeHost(h_cov);                                                 
    cudaFreeHost(h_cov_check);                                           
    cudaFree(d_data);                                                    
    cudaFree(d_mean);                                                   
    cudaFree(d_cov);          

    if(no_errors)
        std::cout << "CUDA programming is fun!" << std::endl;    
                     
}

terminate called after throwing an instance of 'std::runtime_error'
  what():  can't open file /content/celebA_gray_lowres.10000_55_45_32.bin

